In [2]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [4]:
#load data
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")
data_sample

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [5]:
updated_data = [{'Name': item['Name'], 'Symptoms' : item['Symptoms']} for item in data_sample['train']]
df = pd.DataFrame(updated_data)
df

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."
...,...,...
395,Urinary Stones (Kidney Stones),"Severe abdominal or back pain, blood in urine,..."
396,Osteoporosis,"Fragile bones, loss of height over time, back ..."
397,Rheumatoid Arthritis,"Joint pain, stiffness, swelling, fatigue, loss..."
398,Type 1 Diabetes,"Frequent urination, Increased thirst, Weight loss"


In [6]:
#Extracting Symptoms
df['Symptoms'] = df['Symptoms'].apply(lambda x: ', '.join(x.split(', ')))
df

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."
...,...,...
395,Urinary Stones (Kidney Stones),"Severe abdominal or back pain, blood in urine,..."
396,Osteoporosis,"Fragile bones, loss of height over time, back ..."
397,Rheumatoid Arthritis,"Joint pain, stiffness, swelling, fatigue, loss..."
398,Type 1 Diabetes,"Frequent urination, Increased thirst, Weight loss"


In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

In [8]:
device = torch.device("cpu")
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [10]:
BATCH_SIZE = 8

In [11]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [12]:
# Dataset Prep
class LanguageDataset(Dataset):
  def __init__(self, df, tokenizer):
    self.labels = df.columns
    self.data = df.to_dict(orient='records')
    self.tokenizer = tokenizer
    x = self.fittest_max_length(df)
    self.max_length = x

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    x = self.data[idx][self.labels[0]]
    y = self.data[idx][self.labels[1]]
    text = f"{x} | {y}"
    tokens = self.tokenizer.encode_plus(text, max_length=128, padding='max_length', truncation=True, return_tensors='pt')
    return tokens

  def fittest_max_length(self, df):
    max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
    x = 2
    while x< max_length:
      x = x*2
    return x


In [21]:
data_sample = LanguageDataset(df, tokenizer)
data_sample

In [22]:
train_size = int(0.8 * len(data_sample))
val_size = len(data_sample) - train_size
train_data, val_data = random_split(data_sample, [train_size, val_size])

In [29]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)

In [30]:
num_epochs = 8

In [31]:
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0

In [32]:
criterion = nn.CrossEntropyLoss(ignore_index= tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [33]:
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu', 'training_loss', 'validation_loss', 'epoch_duration'])

In [ ]:
# The training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer for the epoch

    # Training
    ## This line tells the model we're in 'learning mode'
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    # Validation
    ## This line below tells the model to 'stop learning'
    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    valid_iterator = tqdm(valid_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(valid_loader)

    end_time = time.time()  # End the timer for the epoch
    epoch_duration_sec = end_time - start_time  # Calculate the duration in seconds

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}  # Add epoch_duration to the dataframe

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(valid_loader)}")

Training Epoch 1/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [08:08<00:00, 12.22s/it, Training Loss=0.52]
Validation Epoch 1/8: 100%|██████████| 10/10 [00:37<00:00,  3.71s/it, Validation Loss=0.528]


Epoch: 1, Validation Loss: 0.65443354845047


Training Epoch 2/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [08:01<00:00, 12.05s/it, Training Loss=0.499]
Validation Epoch 2/8: 100%|██████████| 10/10 [00:37<00:00,  3.71s/it, Validation Loss=0.563]


Epoch: 2, Validation Loss: 0.7440325021743774


Training Epoch 3/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [07:54<00:00, 11.87s/it, Training Loss=0.288]
Validation Epoch 3/8: 100%|██████████| 10/10 [00:36<00:00,  3.69s/it, Validation Loss=0.599]


...

Training Epoch 6/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [07:53<00:00, 11.84s/it, Training Loss=0.0851]
Validation Epoch 6/8: 100%|██████████| 10/10 [00:38<00:00,  3.81s/it, Validation Loss=0.754]


Epoch: 6, Validation Loss: 0.9794331789016724


Training Epoch 7/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [07:54<00:00, 11.86s/it, Training Loss=0.12]
Validation Epoch 7/8: 100%|██████████| 10/10 [00:36<00:00,  3.63s/it, Validation Loss=0.766]


Epoch: 7, Validation Loss: 1.0551016330718994


Training Epoch 8/8 Batch Size: 8, Transformer: distilgpt2:  85%|████████▌ | 34/40 [06:43<01:09, 11.58s/it, Training Loss=0.101]

In [35]:
input_str = "Kidney Failure"
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

output = model.generate(
    input_ids,
    max_length=20,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Kidney Failure | Decreased urine output, fluid retention, fatigue, shortness of breath


In [36]:
torch.save(model, 'SmallMedLM.pt')

In [38]:
torch.save(model, '/content/SmallMedLM.pt')